In [128]:
from selenium import webdriver
import bs4
import re
import pandas as pd
import time
from IPython.core.display import HTML

In [129]:
#open chrome driver
path=r'C:\Users\atphor\Downloads\chromedriver'
driver = webdriver.Chrome(executable_path=path)

driver.get('https://shopee.co.th/')


C:\Users\atpho\AppData\Local\Temp\ipykernel_19516\2035735255.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path)


In [130]:
#input data
i=input ('input 1-12 :')
name='00'+i
print('name excel :',name)
head =round(int(i)/2)
if int(i) <= 1:
    head = 1
index = i

if int(index) % 2 == 0:
    index = 2
else:
    index = 1

#input number page
current_url=driver.current_url  
number_page = input('How much data do you want? (1 page = 60 data) :')
number_page = int(number_page)

input 1-12 :1
name excel : 001
How much data do you want? (1 page = 60 data) :1


In [131]:
#close popup
thai_button = driver.find_element("xpath",'/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
thai_button.click()

In [132]:
close_button = driver.execute_script('return document.querySelector("shopee-banner-popup-stateful").shadowRoot.querySelector("div.shopee-popup__close-btn")')
close_button.click()

type_url='/html/body/div[1]/div/div[2]/div/div/div[3]/div[2]/div[1]/div/div/div[2]/div/div[1]/ul/li['+str(head)+']/div/a['+str(index)+']/div/div[1]/div'

Types_button = driver.find_element("xpath",type_url)
#Types_button = driver.find_element("class",'n-CE6j _8YHYKq')
Types_button.click()
time.sleep(5)

Best_sales = driver.find_element("xpath",'/html/body/div[1]/div/div[2]/div/div[1]/div[4]/div[2]/div/div[1]/div[1]/div[3]')
Best_sales.click()

In [133]:
#zoom out and scraping data
driver.execute_script('document.body.style.zoom="10%"')
time.sleep(10)
data = driver.page_source

soup = bs4.BeautifulSoup(data)

#list all product data
all_product =soup.find_all('div',{'class':"_1yN94N WoKSjC _2KkMCe"})

all_product_list = []
for product in all_product:
    all_product_list.append(product.text)

all_price =soup.find_all('div',{'class':"cbl0HO MUmBjS"})

all_price_list = []
for product in all_price:
    all_price_list.append(product.text)

all_sell =soup.find_all('div',{'class':'x+3B8m wOebCz'})

all_sell_list = []
for product in all_sell:
    all_sell_list.append(product.text)


#removing symbol
all_sell_list =re.findall(r'[\d\.\d]+',str(all_sell_list))#ทำให้ลิสไม่มีตัวหนังสือหรือสัญลักษณ์

#make product dataframe
all_data = pd.DataFrame({
    'Product Name': all_product_list,
    'Price': all_price_list,
    'sold(kpcs./Mouth)': all_sell_list
})

#list url
all_pic =soup.find_all('div',{'class':'usRW1x S8FTqJ'})

all_pic = soup.find_all('img',src=True)
all_pic=all_pic
print('Number of Images: ', len(all_pic))

# select src tag
image_src = [x['src'] for x in all_pic]

#sel only pic
image_src = [x for x in image_src if 61 < len(x)]

#make dataframe
df =pd.DataFrame(image_src,columns=['img'])[0:60]

#sel product
img = image_src[0:60]

df['img'] = img
df = pd.concat([df,all_data],axis=1)

# Converting links to html tags
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

# Rendering the dataframe as HTML table
df.to_html(escape=False, formatters=dict(img=path_to_image_html))

# Rendering the images in the dataframe using the HTML method.
HTML(df.to_html(escape=False,formatters=dict(img=path_to_image_html)))

Number of Images:  138


,img,Product Name,Price,sold(kpcs./Mouth)
0,,[5บ. กดได้ 3 ชิ้น ]🔥[พร้อมส่ง] Munafie พร้อมส่งจากไทยBoxerชาย ผ้านิ่มใส่สบายไม่อึดอัดไม่มีถุงซิป MW#00000,฿12,48.5
1,,（แพ็คเกจส่วนบุคคล）👍ส่งเร็ว 1-2 วัน🌈 Munafie พร้อมส่งจากไทยBoxerชาย ผ้านิ่มใส่สบายไม่อึดอัดไม่มีถุงซิป,฿10,13.1
2,,กางเกงในผู้ชายBoocon เนื้อผ้านุ่มใส่สบาย ขอบไม่ระคายเคือง เนื้อผ้าระบายอากาศได้ดี,฿28 - ฿31,12.3
3,,⚡️F09⚡️ซื้อ10แถม1 กางเกงบ๊อกเซอร์ Boxer S-2XL*ราคาต่อชิ้น*,฿25 - ฿55,11.1
4,,เสื้อยืดสีพื้น คอกลม เกรดพรีเมี่ยม❤️ ใส่สบาย ไม่หด ไม่ต้องรีด ราคาส่ง มีเก็บเงินปลายทาง ❤️ (สินค้าพร้อมจัดส่งทุกวันค่ะ),฿37 - ฿52,10.5
5,,TOP1 กางเกงในชาย MUNAFIE เนื้อผ้านุ่ม มีซองใส่ทุกตัว รุ่นขาเว้า,฿12 - ฿14,10.4
6,,MW036กางเกงในชาย ขาเว้า MUNAFIE เนื้อผ้านิ่มบางเบาไม่มีถุงซิป,฿11,8.7
7,,กางเกงในผู้ชาย ผ้านิ่ม ใส่สบาย ขอบไม่เจ็บ มี 8สีให้เลือก,฿28,6.8
8,,【พร้อมส่ง】Nk กางเกงชั้นในผ้าฝ้ายเอวยางยืดพิมพ์ลายตัวอักษรแฟชั่นผู้ชาย,฿29,6.4
9,,กางเกงชั้นในชาย บ็อกเซอร์MUNAFIE NK-19 Boxer บ็อกเซอร์ชาย ไร้รอยต่อ ขาเหลี่ยม สินค้าพร้อมส่ง,฿11,6.1


In [140]:
#name excel
def hi_var(var_name):
    globals()[var_name]=df  
hi_var(name)
df.to_excel(name+'.xlsx')